In [ ]:
import pandas as pd
import openai


openai.api_key = ''
model = "gpt-3.5-turbo-0125" ## ChatGPT version

file_paths = {
    'asthma_pollution': 'final_data_asthma_pollution.csv'
    'cardiovascular': 'final_data_cardiovascular.csv'
    'dementias_alzheimers': 'final_data_dementias_alzheimers.csv'
    'diabetes': 'final_data_diabetes.csv'
    'mental_health': 'final_data_mental_health.csv'
}
results = []
for category, file_path in file_paths.items():
    df = pd.read_csv(file_path)
    for index, row in df.iterrows():
        main_goal_and_aims = row['main_goal_and_aims']
        questions = row['Questions']
        approaches = row['Approaches']
        findings = row['Findings']

        prompt = (
          f"Using the following details from a {category} research project, "
          f"identify ten key phrases or terms related to the project and provide the corresponding UMLS codes. "
          f"Format the output as 'Keyword (UMLS Code)'. For example, if a keyword is 'asthma', "
          f"and its UMLS code is C0004096, format it as 'asthma (C0004096)'.\n\n"
          f"Main goal and aims:\n{row['main_goal_and_aims']}\n\n"
          f"Questions:\n{row['Questions']}\n\n"
          f"Approaches:\n{row['Approaches']}\n\n"
          f"Findings:\n{row['Findings']}\n"
      )

        messages = [{"role": "system", "content": "Extract keywords from each project and advise keywords with UMLS vocabulary and provide UMLS code. Follow the format in the prompt strictly"},
                {"role": "user", "content": prompt }]

        try:
          response = openai.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=messages
          )
          extracted_info = response.choices[0].message.content
          #print(response)
          results.append(extracted_info)
          #print(f"Original: {row['Team']}\nExtracted: {extracted_info}\n")
          #print(f"Extracted: {extracted_info}\n")
        except openai.OpenAIError as e:  # Correct exception handling
          print(f"An error occurred: {e}")
          results.append(None)


In [ ]:
import pandas as pd
import re

structured_keywords = []
for item in results:
    keywords_dict = {'Keywords': [], 'UMLS_Codes': []}
    lines = item.split('\n')
    for line in lines:
        match = re.search(r'^(?:\d+\.\s*)?(?P<keyword>.+?) \((?P<umls_code>C\d+)\)$', line.strip())
        if match:
            keyword_clean = match.group('keyword').strip(' -')
            umls_code = match.group('umls_code')

            keywords_dict['Keywords'].append(keyword_clean)
            keywords_dict['UMLS_Codes'].append(umls_code)

    if not keywords_dict['Keywords']:
        print(f"Problematic item with no keywords found: {item}")
        structured_keywords.append(keywords_dict)
    else:
        structured_keywords.append(keywords_dict)


In [ ]:
import pandas as pd

# Check if the lengths match
print(len(df))
print(len(structured_keywords))
if len(df) == len(structured_keywords):
    # Add the Keywords and UMLS_Codes to the original DataFrame
    df['Keywords'] = [kw['Keywords'] for kw in structured_keywords]
    df['UMLS_Codes'] = [kw['UMLS_Codes'] for kw in structured_keywords]

    # Save the updated DataFrame to a new CSV file
    df.to_csv('updated_with_keywords.csv', index=False)
else:
    print("The length of the original DataFrame and structured keywords does not match.")


244
244
